In [1]:
import torch, random
import torchvision
from torchvision.datasets import CIFAR10 
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn 
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
import json, string
import numpy as np
from tqdm import tqdm as tqdm
%matplotlib inline


In [84]:
# delete the file that cannot be opened
import os
for path, subdirs, files in os.walk('/home/ningjy/jianyusu/Image_recognition/cs6501_final_project/all_data/'):
    for name in files:
        file = os.path.join(path, name)
        with open(file, 'rb') as f:
            try:
                Image.open(f).convert('RGB')
            except IOError as e:  
                os.remove(file)

KeyboardInterrupt: 

In [12]:
# Read data from my local machine and it works this way
# root/1/xxx.png
# root/1/xxy.png
# root/1/xxz.png

# root/2/123.png
# root/2/nsdf3.png
# root/2/asd932_.png

imgTransform = transforms.Compose([transforms.Scale(128,128),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), 
                                                        (0.2023, 0.1994, 0.2010))])

CFTrainSet = torchvision.datasets.ImageFolder(root = "./all_data/train", imgTransform)
CFValSet = torchvision.datasets.ImageFolder(root = "./all_data/val", imgTransform)

CFTrainLoader = torch.utils.data.DataLoader(CFTrainSet, batch_size = 64, 
                                          shuffle = True, num_workers = 4)
CFValLoader = torch.utils.data.DataLoader(CFValSet, batch_size = 64, 
                                          shuffle = True, num_workers = 4)

In [ ]:
for ele in CFTrainSet:
    if ele[0].size(1) != 128 or ele[0].size(2) != 128 or ele[0].size(0) != 3:
        print(ele[0].size())

In [8]:
model = torchvision.models.vgg11_bn(pretrained=True)

# Feel free to use a different loss here.
criterion = nn.CrossEntropyLoss()

learningRate = 1e-2

# Feel free to change the optimizer, or the optimizer parameters. e.g. momentum, weightDecay, etc.
optimizer = optim.SGD(model.parameters(), lr = learningRate)

In [9]:
# Train the previously defined model.
def train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 10, use_gpu = False):
    train_acc = list()
    val_acc = list()
    train_loss = list()
    val_loss = list()
    if use_gpu:
        network = network.cuda()
        criterion = criterion.cuda()
        
    # Training loop.
    for epoch in range(0, n_epochs):
        correct = 0.0
        cum_loss = 0.0
        counter = 0

        # Make a pass over the training data.
        t = tqdm(trainLoader, desc = 'Training epoch %d' % epoch)
        network.train()  # This is important to call before training!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # Backward pass:
            optimizer.zero_grad()
            # Loss is a variable, and calling backward on a Variable will
            # compute all the gradients that lead to that Variable taking on its
            # current value.
            loss.backward() 

            # Weight and bias updates.
            optimizer.step()

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
        
        train_loss.append(cum_loss / (1 + i))
        train_acc.append(100 * correct / counter)
        torch.save({
            'epoch': epoch + 1,
            'state_dict': network.state_dict(),
        }, 'checkpoint.tar' )

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        t = tqdm(valLoader, desc = 'Validation epoch %d' % epoch)
        network.eval()  # This is important to call before evaluating!
        for (i, (inputs, labels)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            labels = Variable(labels)
            
            if use_gpu:
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, labels)

            # logging information.
            cum_loss += loss.data[0]
            max_scores, max_labels = outputs.data.max(1)
            correct += (max_labels == labels.data).sum()
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
            
        val_loss.append(cum_loss / (1 + i))
        val_acc.append(100 * correct / counter)
        
        epochs = np.arange(0.0, n_epochs, 1.0)

    plt.figure(1)
    plt.subplot(211)
    plt.plot(epochs, train_acc, '-o', label = 'train_acc' )
    plt.plot(epochs, val_acc, '-o', label = 'validate_acc')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')

    plt.subplot(212)
    plt.plot(epochs, train_loss, '-o', label = 'train_loss')
    plt.plot(epochs, val_loss, '-o', label = 'validate_loss')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.tight_layout()
    plt.show()
        

In [13]:

train_model(model, criterion, optimizer, CFTrainLoader, CFValLoader, n_epochs = 10, use_gpu = True)

Training epoch 0:   0%|          | 0/2159 [00:00<?, ?it/s]

ValueError: Traceback (most recent call last):
  File "/home/ningjy/anaconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ningjy/anaconda2/lib/python2.7/site-packages/torchvision-0.1.9-py2.7.egg/torchvision/datasets/folder.py", line 118, in __getitem__
    img = self.transform(img)
  File "/home/ningjy/anaconda2/lib/python2.7/site-packages/torchvision-0.1.9-py2.7.egg/torchvision/transforms.py", line 34, in __call__
    img = t(img)
  File "/home/ningjy/anaconda2/lib/python2.7/site-packages/torchvision-0.1.9-py2.7.egg/torchvision/transforms.py", line 197, in __call__
    return img.resize((ow, oh), self.interpolation)
  File "/home/ningjy/.local/lib/python2.7/site-packages/PIL/Image.py", line 1722, in resize
    raise ValueError("unknown resampling filter")
ValueError: unknown resampling filter
